In [34]:
import pandas as pnd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [35]:
!pip install xlrd

You should consider upgrading via the '/Users/alex/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


Cargamos el csv para empezar el trabajo

In [36]:
mensajesTwitter = pnd.read_csv("datas/calentamientoClimatico_sin_preparar.csv", delimiter=";")
mensajesTwitter.head(10)

,Column1,Column2,Column3
0,tweet,existence,existence.confidence
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
6,RT @sejorg: RT @JaymiHeimbuch: Ocean Saltiness...,Yes,1
7,Global warming evidence all around us|A messag...,Yes,1
8,Migratory Birds' New Climate Change Strategy: ...,Yes,1
9,Southern Africa: Competing for Limpopo Water: ...,Yes,1


Vemos la dimensión del dataset

In [37]:
mensajesTwitter.shape

(6091, 3)

Cambiamos el nombre de la columnas para reconocer cada una de ellas

In [38]:
column_names=['TWEET', 'CREENCIA', 'CONFIANZA']
mensajesTwitter.columns = column_names
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
0,tweet,existence,existence.confidence
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
...,...,...,...
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6087,Virginia to Investigate Global Warming Scienti...,NaN,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Miramos a ver si tenemos algun valor nulo

In [39]:
mensajesTwitter.isna().sum()

TWEET           0
CREENCIA     1865
CONFIANZA       3
dtype: int64

In [40]:
mensajesTwitter = mensajesTwitter.dropna()
mensajesTwitter

,TWEET,CREENCIA,CONFIANZA
0,tweet,existence,existence.confidence
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
...,...,...,...
6085,It's 83�_� and climbing in NYC. August weather...,Yes,1
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1


Verificamos, una vez realizada la limpieza de nulos, que ya no queda ninguno

In [41]:
mensajesTwitter.isna().sum()

TWEET        0
CREENCIA     0
CONFIANZA    0
dtype: int64

Eliminamos la primera fila que no tenía ninguna utilidad

In [42]:
mensajesTwitter.drop([0],axis=0)

,TWEET,CREENCIA,CONFIANZA
1,Global warming report urges governments to act...,Yes,1
2,Fighting poverty and global warming in Africa ...,Yes,1
3,Carbon offsets: How a Vatican forest failed to...,Yes,0.8786
4,Carbon offsets: How a Vatican forest failed to...,Yes,1
5,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes,0.8087
...,...,...,...
6085,It's 83�_� and climbing in NYC. August weather...,Yes,1
6086,"@bloodless_coup ""The phrase 'global warming' s...",Yes,1
6088,Global warming you tube parody you will enjoy ...,No,0.6411
6089,One-Eyed Golfer: Don't dare tell me about glob...,No,1
